# **FINAL CAPSTONE PROJECT IN COURSERA DATA SCIENCE**

by : Resha Mochamad Ilham

## Introduction / Problem Description

COVID-19 has been disrupting the life of so many people, while we in Indonesia will start to ease the lockdown in the next few weeks, but the case of the COVID still alarmingIn this notebook, I will analyze the case of  COVID-19 in Jakarta, Indonesia. My intention is to build a script that will monitoring confirmed cases in Jakarta via Health Oficial Site, mapping it base on kelurahan / neighborhood and kecamatan / borough. I also will utilize foursquare to find healthcare facilities that located in the borough to assess the density of health care facilities.

## Data 

#### The source of data that will be use in this notebook are
1. Formal report of COVID-19 case in Jakarta will be downloaded from:  
https://riwayat-file-covid-19-dki-jakarta-jakartagis.hub.arcgis.com/
2. List of hospital will utilize Foursquare data

## Methodology

### This step will be use for the assessment
1. Load the Jakarta confirmed case, including detail location
2. Process the data to get, information for COVID-19
3. Clustering the Neighborhood 
4. Find the highest fatality region
5. Utilize Foursquare API for find nearest hospital
6. Mapping it

#### Data Processing

In [77]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [110]:
# Read excel data form local file
df = pd.read_excel('Data_Covid.xlsx')

# Change column name to englis language
df.rename(columns = {'ID_KEL':'ID_Neighborhood', 
                     'ID_KEL.1':'ID_Neighborhood1',
                     'Nama_provinsi':'State',
                     'nama_kota':'City',
                     'nama_kecamatan':'Borough',
                     'nama_kelurahan':'Neighborhood',
                     'Proses Pemantauan':'Active ODP',
                    'Selesai Pemantauan':'Solved ODP',
                    'Masih Dirawat':'Active PDP',
                    'Pulang dan Sehat':'Solved PDP',
                    'POSITIF':'Confirmed Case',
                    'Dirawat':'Hospitalization',
                    'Sembuh':'Recovered',
                    'Meninggal':'Death'}, inplace = True)

# Delete Column ID_Neigborhood1 due to have same value of Column ID_Neigborhood                  
# Delete Column Keterangan due to not have value
df.drop(['Keterangan','ID_Neighborhood1'], axis=1, inplace=True)

# Delete First Row because it's a Total Case
# Delete Second Row because it's show unknown case
# Delete Third Row because it's show a case in outside Jakarta area
df.drop(df.index[[0,1,2]],inplace=True)

# Sort the dataframe
df_sort = df.sort_values(by =['City','Borough','Neighborhood'])
df_sort.head()

,ID_Neighborhood,State,City,Borough,Neighborhood,ODP,Active ODP,Solved ODP,PDP,Active PDP,Solved PDP,Confirmed Case,Hospitalization,Recovered,Death,Self Isolation
142,3173011001,DKI JAKARTA,JAKARTA BARAT,CENGKARENG,CENGKARENG BARAT,178,1,177,17,1,16,31,10,5,0,16
64,3173011006,DKI JAKARTA,JAKARTA BARAT,CENGKARENG,CENGKARENG TIMUR,107,0,107,23,5,18,38,10,9,2,17
51,3173011002,DKI JAKARTA,JAKARTA BARAT,CENGKARENG,DURI KOSAMBI,78,0,78,18,3,15,17,6,7,1,3
93,3173011005,DKI JAKARTA,JAKARTA BARAT,CENGKARENG,KAPUK,162,1,161,23,5,18,25,4,9,6,6
159,3173011004,DKI JAKARTA,JAKARTA BARAT,CENGKARENG,KEDAUNG KALI ANGKE,46,0,46,8,2,6,13,5,1,0,7


In [111]:
# Get an update regarding total of confirmed case, hospitalization, self isolation and death
Total = df.sum(axis = 0, skipna = True)
print('Currently there are {} confirmed case, {} case still in hospital, {} case of self isolation, and {} person die.'.format(Total[11],Total[12],Total[15],Total[14]))

Currently there are 5178 confirmed case, 1247 case still in hospital, 1965 case of self isolation, and 441 person die.


In [179]:
# define the dataframe columns
column_names = ['Borough','Neighborhood','Confirmed Case', 'Death','Fatality Rate'] 

# instantiate the dataframe
Region = pd.DataFrame(columns=column_names)
Region.head()

,Borough,Neighborhood,Confirmed Case,Death,Fatality Rate


In [180]:
# Get latitude, longitude of the Jakarta Neighborhood
for x in range(len(df_sort)):
    Kel = df_sort.iat[x,4]
    Kec = df_sort.iat[x,3]
    CC = df_sort.iat[x,11]
    De = df_sort.iat[x,14]
    FR = 100*(df_sort.iat[x,14]/df_sort.iat[x,11])
    Region = Region.append({'Borough': Kec,'Neighborhood' : Kel ,'Confirmed Case':CC, 'Death': De,'Fatality Rate':FR} , ignore_index=True)
else:
  print("Finally finished!")
Region["Fatality Rate"] = Region["Fatality Rate"].fillna(0)
Region.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys


Finally finished!


,Borough,Neighborhood,Confirmed Case,Death,Fatality Rate
0,CENGKARENG,CENGKARENG BARAT,31,0,0.000000
1,CENGKARENG,CENGKARENG TIMUR,38,2,5.263158
2,CENGKARENG,DURI KOSAMBI,17,1,5.882353
3,CENGKARENG,KAPUK,25,6,24.000000
4,CENGKARENG,KEDAUNG KALI ANGKE,13,0,0.000000


In [181]:
# Sorting the region based on Fatality Rate
Region_sort = Region.sort_values(by =['Fatality Rate'], ascending = False)
Region_sort.head()

,Borough,Neighborhood,Confirmed Case,Death,Fatality Rate
153,SETIA BUDI,KARET SEMANGGI,3,2,66.666667
101,CILANDAK,CIPETE SELATAN,6,3,50.000000
37,TAMAN SARI,GLODOK,2,1,50.000000
144,PASAR MINGGU,RAGUNAN,6,2,33.333333
124,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,13,4,30.769231


In [189]:
#get latitude and longitude of Jakarta Province
Alamat = 'Jakarta'
geolocator = Nominatim()
location_jak = geolocator.geocode(Alamat)
latitude_jak = location_jak.latitude
longitude_jak = location_jak.longitude
print('The geograpical coordinate of Jakarta are {}, {}.'.format(latitude_jak, longitude_jak))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Jakarta are -6.1753942, 106.827183.


In [230]:
# define the dataframe columns
top_names = ['Borough','Neighborhood','Latitude','Longitude','Fatality Rate'] 

# instantiate the dataframe
Top = pd.DataFrame(columns=top_names)
Top.head()

,Borough,Neighborhood,Latitude,Longitude,Fatality Rate


In [231]:
# Find 50 Location data of region with highest fatality rate
n = 50
for i in range (n):
    Kel1 = Region_sort.iat[i,1]
    Kec1 = Region_sort.iat[i,0]
    geolocator = Nominatim()
    location = geolocator.geocode(Kel1)
    latitude = location.latitude
    longitude = location.longitude
    FR = Region_sort.iat[i,4]
    #print('The geograpical coordinate of {} are {}, {}.'.format(Kel1,latitude, longitude))
    Top = Top.append({'Borough': Kec1,'Neighborhood' : Kel1 ,'Latitude':latitude, 'Longitude': longitude,'Fatality Rate':FR} , ignore_index=True)
else:
  print("Finally finished!")
Top.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


Finally finished!


,Borough,Neighborhood,Latitude,Longitude,Fatality Rate
0,SETIA BUDI,KARET SEMANGGI,-6.222896,106.817882,66.666667
1,CILANDAK,CIPETE SELATAN,-6.272540,106.805768,50.000000
2,TAMAN SARI,GLODOK,-6.144709,106.813092,50.000000
3,PASAR MINGGU,RAGUNAN,-6.296882,106.821011,33.333333
4,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,-6.253975,106.777799,30.769231


In [239]:
# create map of Jakarta using latitude and longitude values
map_jak = folium.Map(location=[latitude_jak, longitude_jak], zoom_start=11)

# add markers to map indicate the region
for lat, lng, neighborhood in zip(Top['Latitude'], Top['Longitude'], Top['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jak)  
    
map_jak

In [214]:
Top.tail()

,Borough,Neighborhood,Latitude,Longitude,Fatality Rate
55,TEBET,BUKIT DURI,-6.220104,106.856599,16.666667
56,MATRAMAN,KEBON MANGGIS,-6.206348,106.856374,16.666667
57,MENTENG,CIKINI,-6.191406,106.840034,16.666667
58,TANJUNG PRIOK,PAPANGGO,-6.129975,106.871970,16.666667
59,GROGOL PETAMBURAN,JELAMBAR BARU,-6.149514,106.786717,16.666667


In [232]:
# Find the other data of COVID-19
df_maxODP=df_sort.loc[df_sort['ODP'].idxmax()]
df_maxPDP=df_sort.loc[df_sort['PDP'].idxmax()]
df_maxCC=df_sort.loc[df_sort['Confirmed Case'].idxmax()]
df_maxH=df_sort.loc[df_sort['Hospitalization'].idxmax()]
df_maxSI=df_sort.loc[df_sort['Self Isolation'].idxmax()]
df_maxD=df_sort.loc[df_sort['Death'].idxmax()]

In [233]:
# then, print it out
print('The area with highest ODP are {},{}.'.format(df_maxODP['Neighborhood'],df_maxODP['Borough']))
print('The area with highest PDP are {},{}.'.format(df_maxPDP['Neighborhood'],df_maxPDP['Borough']))
print('The area with highest Confirmed Case are {},{}.'.format(df_maxCC['Neighborhood'],df_maxCC['Borough']))
print('The area with highest Hospitalization are {},{}.'.format(df_maxH['Neighborhood'],df_maxH['Borough']))
print('The area with highest Self Isolation are {},{}.'.format(df_maxSI['Neighborhood'],df_maxSI['Borough']))
print('The area with highest Death are {},{}.'.format(df_maxD['Neighborhood'],df_maxD['Borough']))

The area with highest ODP are PADEMANGAN BARAT,PADEMANGAN.
The area with highest PDP are SUNTER AGUNG,TANJUNG PRIOK.
The area with highest Confirmed Case are SUNTER AGUNG,TANJUNG PRIOK.
The area with highest Hospitalization are SUNTER AGUNG,TANJUNG PRIOK.
The area with highest Self Isolation are PETAMBURAN,TANAH ABANG.
The area with highest Death are SUNTER AGUNG,TANJUNG PRIOK.


In [234]:
# Data processing for clustering
from sklearn.preprocessing import StandardScaler

X = Top.values[:,2:4]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [235]:
# Clustring the data
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[2 2 0 2 2 0 2 2 0 0 2 0 0 0 0 0 0 2 0 1 0 2 2 2 2 2 0 2 0 2 2 2 0 0 0 0 2
 2 2 2 0 2 0 0 0 2 0 0 0 0]


In [236]:
# Add label to the data
Top["Labels"] = labels
Top.head(5)

,Borough,Neighborhood,Latitude,Longitude,Fatality Rate,Labels
0,SETIA BUDI,KARET SEMANGGI,-6.222896,106.817882,66.666667,2
1,CILANDAK,CIPETE SELATAN,-6.272540,106.805768,50.000000,2
2,TAMAN SARI,GLODOK,-6.144709,106.813092,50.000000,0
3,PASAR MINGGU,RAGUNAN,-6.296882,106.821011,33.333333,2
4,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,-6.253975,106.777799,30.769231,2


In [245]:
Top.groupby('Labels').mean()

,Latitude,Longitude,Fatality Rate
Labels,,,
0,-6.160720,106.854861,22.662082
1,-7.550515,110.819406,25.000000
2,-6.263514,106.820719,25.499149


In [279]:
# Get Location data of the Highest Fatality Rate
Lat_top=Top.iat[0,2]
Lon_top=Top.iat[0,3]
Lat_top


-6.2228956

In [252]:
# Prepare for Foursquare connection

CLIENT_ID = '3IEPRVQVJXUILY1S243R3WE2EH5D4OX331ZCCT4FZZDW3GCP' # your Foursquare ID
CLIENT_SECRET = 'SVSKRR3BAUH115LZEXJHMGG5E3IRG1Z3HLOZMO5UNFTCVFNE' # your Foursquare Secret
VERSION = '20200530'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

search_query = 'Hospital'
radius = 500
print(search_query + ' .... OK!')

Your credentails:
CLIENT_ID: 3IEPRVQVJXUILY1S243R3WE2EH5D4OX331ZCCT4FZZDW3GCP
CLIENT_SECRET:SVSKRR3BAUH115LZEXJHMGG5E3IRG1Z3HLOZMO5UNFTCVFNE
Hospital .... OK!


In [254]:
# URL with search querry of Hospital
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Lat_top, Lon_top, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3IEPRVQVJXUILY1S243R3WE2EH5D4OX331ZCCT4FZZDW3GCP&client_secret=SVSKRR3BAUH115LZEXJHMGG5E3IRG1Z3HLOZMO5UNFTCVFNE&ll=-6.2228956,106.81788225863943&v=20200530&query=Hospital&radius=500&limit=30'

In [255]:
# Show the result 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed3c157949393001cc967e7'},
 'response': {'venues': [{'id': '4d09ec788ea0a143341fe373',
    'name': 'MRCCC Siloam Hospitals',
    'location': {'address': 'Jalan Garnisun Dalam No. 2-3',
     'crossStreet': 'Semanggi',
     'lat': -6.21894485966554,
     'lng': 106.81718750941442,
     'labeledLatLngs': [{'label': 'display',
       'lat': -6.21894485966554,
       'lng': 106.81718750941442}],
     'distance': 446,
     'postalCode': '12930',
     'cc': 'ID',
     'city': 'Jakarta Selatan',
     'state': 'Jakarta',
     'country': 'Indonesia',
     'formattedAddress': ['Jalan Garnisun Dalam No. 2-3 (Semanggi)',
      'Jakarta Selatan',
      'Jakarta 12930',
      'Indonesia']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}

In [256]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4d09ec788ea0a143341fe373,MRCCC Siloam Hospitals,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1590935912,False,Jalan Garnisun Dalam No. 2-3,Semanggi,-6.218945,106.817188,"[{'label': 'display', 'lat': -6.21894485966554...",446,12930,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jalan Garnisun Dalam No. 2-3 (Semanggi), Jaka..."
1,511308a9e4b00a46913cee89,"Hospital,singapore","[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1590935912,False,NaN,NaN,-6.220140,106.819880,"[{'label': 'display', 'lat': -6.22013962106914...",378,NaN,ID,NaN,Singapore,Indonesia,"[Singapore, Indonesia]"
2,5c92f7bc6eda02002c86a992,"Mrccc Siloam Hospital 32Nd Floor, Room 3210","[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1590935912,False,Jalan Garnisun Dalam kav. 2-3\nSemanggi,NaN,-6.219046,106.817160,"[{'label': 'display', 'lat': -6.219046, 'lng':...",435,12930,ID,South Jakarta,Jakarta,Indonesia,"[Jalan Garnisun Dalam kav. 2-3, Semanggi, Sout..."
3,4cad689a2a55224b0f1d6e69,Dental Clinic - Jakarta Hospital,"[{'id': '4bf58dd8d48988d178941735', 'name': 'D...",v-1590935912,False,Jalan Jendral Sudirman No. 12,NaN,-6.218243,106.816717,"[{'label': 'display', 'lat': -6.21824293171970...",533,10270,ID,Jakarta,Jakarta,Indonesia,"[Jalan Jendral Sudirman No. 12, Jakarta, Jakar..."
4,514cb9f4e4b0692dc98a3bfa,819 8 th Floor Medistra Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1590935912,False,Jl. Gatot Subroto,NaN,-6.225238,106.816605,"[{'label': 'display', 'lat': -6.225238, 'lng':...",296,NaN,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jl. Gatot Subroto, Jakarta Selatan, Jakarta, ..."


In [257]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,MRCCC Siloam Hospitals,Hospital,Jalan Garnisun Dalam No. 2-3,Semanggi,-6.218945,106.817188,"[{'label': 'display', 'lat': -6.21894485966554...",446,12930,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jalan Garnisun Dalam No. 2-3 (Semanggi), Jaka...",4d09ec788ea0a143341fe373
1,"Hospital,singapore",Hospital,NaN,NaN,-6.220140,106.819880,"[{'label': 'display', 'lat': -6.22013962106914...",378,NaN,ID,NaN,Singapore,Indonesia,"[Singapore, Indonesia]",511308a9e4b00a46913cee89
2,"Mrccc Siloam Hospital 32Nd Floor, Room 3210",Hospital,Jalan Garnisun Dalam kav. 2-3\nSemanggi,NaN,-6.219046,106.817160,"[{'label': 'display', 'lat': -6.219046, 'lng':...",435,12930,ID,South Jakarta,Jakarta,Indonesia,"[Jalan Garnisun Dalam kav. 2-3, Semanggi, Sout...",5c92f7bc6eda02002c86a992
3,Dental Clinic - Jakarta Hospital,Dentist's Office,Jalan Jendral Sudirman No. 12,NaN,-6.218243,106.816717,"[{'label': 'display', 'lat': -6.21824293171970...",533,10270,ID,Jakarta,Jakarta,Indonesia,"[Jalan Jendral Sudirman No. 12, Jakarta, Jakar...",4cad689a2a55224b0f1d6e69
4,819 8 th Floor Medistra Hospital,Hospital,Jl. Gatot Subroto,NaN,-6.225238,106.816605,"[{'label': 'display', 'lat': -6.225238, 'lng':...",296,NaN,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jl. Gatot Subroto, Jakarta Selatan, Jakarta, ...",514cb9f4e4b0692dc98a3bfa
5,"Siloam Hospital, 25th FLOOR",Hospital,"Jalan Garnisun Dalam 8, Jakarta Capital Region...",NaN,-6.223121,106.815202,"[{'label': 'display', 'lat': -6.22312095316648...",297,NaN,ID,Jakarta,Jakarta,Indonesia,"[Jalan Garnisun Dalam 8, Jakarta Capital Regio...",4eebf2b79adf257d65139007
6,MRCCC SILOAM HOSPITAL,None,NaN,NaN,-6.223842,106.820719,"[{'label': 'display', 'lat': -6.223842, 'lng':...",331,NaN,ID,NaN,NaN,Indonesia,[Indonesia],55cb12ad498e6c04b92effb8
7,Ruang Mandi Jenazah Siloam Hospital,None,NaN,NaN,-6.219967,106.816378,"[{'label': 'display', 'lat': -6.219967, 'lng':...",366,NaN,ID,NaN,NaN,Indonesia,[Indonesia],55b9f7db498ececd4a8f88e7
8,ASKES CENTER - MRCCC Siloam Hospital Semanggi,Hospital,Jl.Garnisun Dalam Kav.2-3,Karet Setiabudi,-6.219435,106.817777,"[{'label': 'display', 'lat': -6.21943475487736...",385,12930,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jl.Garnisun Dalam Kav.2-3 (Karet Setiabudi), ...",50811007e4b0921f5b3451fa
9,Ladies Locker Room Mrccc Siloam Hospital,None,NaN,NaN,-6.219222,106.817279,"[{'label': 'display', 'lat': -6.219222, 'lng':...",414,NaN,ID,NaN,NaN,Indonesia,[Indonesia],532bdf90498eac26d6798e1f


In [273]:
# Take the location data of the nearest hospital
List_hospital = dataframe.dropna()
Lat_Hos=List_hospital.iat[0,7]
Lon_Hos=List_hospital.iat[0,8]
List_hospital.head(1)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4d09ec788ea0a143341fe373,MRCCC Siloam Hospitals,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1590935912,False,Jalan Garnisun Dalam No. 2-3,Semanggi,-6.218945,106.817188,"[{'label': 'display', 'lat': -6.21894485966554...",446,12930,ID,Jakarta Selatan,Jakarta,Indonesia,"[Jalan Garnisun Dalam No. 2-3 (Semanggi), Jaka..."


In [283]:
# Mapping it out
venues_map = folium.Map(location=[Lat_top, Lon_top], zoom_start=13) 
# add a red circle marker 
folium.features.CircleMarker(
    [Lat_top, Lon_top],
    radius=10,
    color='red',
    popup='Top Covid Case',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Nearest Hospital
folium.features.CircleMarker(
    [Lat_Hos, Lon_Hos],
    radius=10,
    color='red',
    popup='Nearest Hospital',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)
# display map

venues_map

## Result

Based on the data, its show that some of the hit hard impacted region in Jakarta are spread in all of region. The hardest hit region even in the downtown of the city.  The region is very near to the hospital, other parameter should be consider why the fatality in this region its high. The data foursquare generated in Jakarta area its not quite good, a lot of duplication, non significant building, and not accurate distance.

## Discussion

Other parameter that should be try, including  
1. Highest Confirmed Case region
2. Data of State of Health Care Centre instead of Hospital, but unfortunately Foursquare not provide this

## Conclusion

The script is good, and the result is suitable in the real condition